# Fetch decidim data (API and Open Data)

In [1]:
import pandas as pd
import requests
import json
from urllib.parse import urlparse
import zipfile
import glob
import shutil

In [2]:
query = '{ participatoryProcess(slug: "XXX") { id slug hashtag type createdAt updatedAt startDate endDate heroImage bannerImage reference promoted showMetrics scopesEnabled title { translations { locale text } } subtitle { translations { locale text } } description { translations { locale text } } shortDescription { translations { locale text } } developerGroup { translations { locale text } } announcement { translations { locale text } } participatoryScope { translations { locale text } } participatoryStructure { translations { locale text } } linkedParticipatorySpaces { id name } categories { id name { translations { locale text } } } scope { id name { translations { locale text } } children { id name { translations { locale text } } children { id name { translations { locale text } } children { id name { translations { locale text } } } } } } metaScope { translations { locale text } } localArea { translations { locale text } } stats { name value } steps { position title { translations { locale text } } startDate endDate } target { translations { locale text } } attachments { url type thumbnail } components { id __typename name { translations { text } } weight } } decidim { version applicationName } }'
query_min = '{ participatoryProcess(id: "XXX") { id slug hashtag type createdAt updatedAt startDate endDate heroImage bannerImage reference promoted scopesEnabled title { translations { locale text } } subtitle { translations { locale text } } description { translations { locale text } } shortDescription { translations { locale text } } developerGroup { translations { locale text } } announcement { translations { locale text } } participatoryScope { translations { locale text } } participatoryStructure { translations { locale text } } scope { id name { translations { locale text } } children { id name { translations { locale text } } children { id name { translations { locale text } } children { id name { translations { locale text } } } } } } metaScope { translations { locale text } } localArea { translations { locale text } } stats { name value } steps { title { translations { locale text } } startDate endDate } target { translations { locale text } } attachments { url type thumbnail } components { id __typename name { translations { text } } weight } } decidim { version applicationName } }'

df = pd.read_csv('decidimpb.csv',sep='\t')
processes = df[['PB process url','PB process id']].values

for process in processes:
    print (process)
    id = str(int(process[1]))
    host = process[0].split('/processes/')[0]
    process = process[0].split('/processes/')[1]
    if True:
    #try:
        parsed_uri = urlparse(host)
        data = requests.post(host+"/api", data = {'query': query.replace('XXX',process)}, verify=False).json()
        if 'errors' in data:
            data = requests.post(host+"/api", data = {'query': query_min.replace('XXX',id)}, verify=False).json()
        with open('processes/'+ '{uri.netloc}'.format(uri=parsed_uri).replace('.','_')+'__'+process+'.json', 'w') as outfile:
            json.dump(data, outfile, indent=4)
    #except Exception as e:
    #    print('error api',host,process,e)
    try:
        r = requests.get(host+'/open-data/download', allow_redirects=True, stream=True, verify=False)
        path = 'open-data/'+ '{uri.netloc}'.format(uri=parsed_uri).replace('.','_')
        with open(path+'.zip', 'wb') as fd:
            fd.write(r.content) 
        with zipfile.ZipFile(path+'.zip',"r") as zip_ref:
            zip_ref.extractall(path)

        csv_files = (glob.glob(path+'/*'))
        for csv_file in csv_files:
            if csv_file.endswith('meetings.csv') or csv_file.endswith('proposals.csv'):
                try:
                    df_csv = pd.read_csv(csv_file,sep=';')
                    df_csv = df_csv[df_csv['participatory_space/url']==host+'/processes/'+process+'?participatory_process_slug='+process]
                    df_csv.to_csv(csv_file.split('-')[-1].split('.')[0]+'/'+path.split('/')[-1]+process+'.csv', index=False, header=True)
                except Exception as e2:
                    print('error csv',csv_file, e2)
        shutil.rmtree(path)
    except Exception as e1:
        print('error open data',host,process,e1)
        
    print ('\n')

['https://decidim-calaf.diba.cat/processes/pressupostosparticipatiuscalaf2020'
 216]


/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'decidim-calaf.diba.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'decidim-calaf.diba.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'decidim.diba.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(




['https://decidim.calafell.cat/processes/pressupostos-participatius-2020'
 21]


/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'decidim.calafell.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'decidim.calafell.cat'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'decidim-calafell.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(




['https://participa.gavaciutat.cat/processes/Participaccio' 23]


...

/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'atelierdevosidees.loiret.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'atelierdevosidees.loiret.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'atelierdevosidees.loiret.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn

error csv open-data/atelierdevosidees_loiret_fr/atelierdevosidees.loiret.fr-open-data-meetings.csv No columns to parse from file


['https://budgetparticipatif.saintjeandebraye.fr/processes/bp' 1]


/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'budgetparticipatif.saintjeandebraye.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'budgetparticipatif.saintjeandebraye.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'budgetparticipatif.saintjeandebraye.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.



['https://budgetparticipatif.saintjeandebraye.fr/processes/bp2019-2020' 2]


/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'budgetparticipatif.saintjeandebraye.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'budgetparticipatif.saintjeandebraye.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/paragon/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'budgetparticipatif.saintjeandebraye.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.